In [29]:
import datetime
from pathlib import Path
import pickle

import numpy as np
import pandas as pd
from bisect import bisect_right
import torch

from configurations import RMV_NON_UNIQUE_TIMESTAMP, NON_UNIQUE_TIMESTAMP_INDEX_RANGE, TRAIN_DATA_BORDER, TEST_DATA_BORDER
import macros

In [67]:
with open('./cache/preprocessed/df000.pickle', 'rb') as fp:
    train_df, test_df, full_sen_list, df_status = pickle.load(fp)

In [68]:
full_sen_list

array([ 0,  1,  2,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
      dtype=uint8)

In [6]:
from configurations import SEQ_LEN
full_sensor_list = full_sen_list
def extract_features(df, error_sensor_timestamps_dict):
    sen_num = len(full_sensor_list)
    dfs = []
    for sen in full_sensor_list:
        diff_df = df[df.sensor==sen].diff() # on~off and off~on time
        diff_df = diff_df[diff_df.event == -1] # on~off time only
        diff_ser = diff_df.timestamp.reset_index(drop=True)
        diff_flt = diff_ser.apply(datetime.timedelta.total_seconds)
        true_df = df[(df.sensor==sen) & (df.event)].copy().reset_index(drop=True)
        true_df['timediff'] = diff_flt
        true_df.set_index('timestamp', inplace=True)
        base_np = np.zeros((true_df.shape[0], sen_num+1))
        feature_df = pd.DataFrame(base_np,
                columns=[*full_sensor_list, 'timediff'],
                index=true_df.index
            )
        feature_df.loc[:, sen] = true_df.timediff
        dfs.append(feature_df)
    concated = pd.concat(dfs, axis=0).sort_index()
    concated = concated[~concated.isna().any(axis=1)]
    concated['timediff'] = concated.index.to_series().diff().iloc[1:].apply(datetime.timedelta.total_seconds)
    concated = concated.iloc[1:]
    idxs = concated.index
    base_np = np.zeros((concated.shape[0], len(error_sensor_timestamps_dict)))
    error_counts_df = pd.DataFrame(base_np,
            columns=error_sensor_timestamps_dict.keys(),
            index=idxs
        )
    for error_sensor, times in error_sensor_timestamps_dict.items():
        for timestamp in times:
           idx = bisect_right(idxs, timestamp)
           error_counts_df.iloc[idx].loc[error_sensor] += 1
    error_cum = error_counts_df.cumsum(axis=0)
    error_cum = error_cum.iloc[SEQ_LEN-1:].reset_index(drop=True) - \
        error_cum[:-100].shift(1,fill_value=0).reset_index(drop=True)
    error_cum = error_cum[:-1]
    error_cum.set_index(error_counts_df.index[:-SEQ_LEN], drop=True, inplace=True)
    return concated, error_cum

In [63]:
with open('temp.pickle', 'rb') as fp:
    X = pickle.load(fp)

In [65]:
from functions import generate_scaler
scaler = generate_scaler('MinMax')

In [66]:
scaler.fit_transform(X)

ValueError: Found array with dim 3. MinMaxScaler expected <= 2.

In [54]:
with open('temp.pickle', 'rb') as fp:
    label_scores = pickle.load(fp)

In [55]:
labels, preds= zip(*label_scores)

In [58]:
scores = torch.sigmoid(torch.tensor(preds))
scores = scores.data.numpy()
labels = np.array(labels)
labels


array([[1., 1., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [1., 1., 0.],
       [0., 1., 1.],
       [0., 1., 1.]], dtype=float32)

In [62]:
np.abs(scores - labels).sum()

994.0675

In [53]:
from sklearn.metrics import roc_auc_score
idx = 2
auc = roc_auc_score(labels[:,idx], scores[:,idx])
auc

0.6774543124999999